#LOAD LIBRARIES

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Load Libraries
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd 
import chardet
import multiprocessing
import random

from sklearn.preprocessing import scale
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
import timeit

#DATASET

##Read Dataset

Let's read the dataset to use.

In [0]:
df = pd.read_csv('/content/drive/My Drive/first_final_infection_dataset.csv')

In [6]:
df

,fiebre,tos_seca,fatiga,flemas_esputos,dif_resp,dolor_garganta,dolor_cabeza,dolor_muscular,escalofrios,nausea_vomito,cong_nasal,diarrea,expect_sangre,conjuntivitis,dermatitis,falta_gusto_olfato,proteccion_no,proteccion_si,transporte_no,transporte_si,contacto_directo_no,contacto_directo_si,exposicion_alta,exposicion_baja,exposicion_media,zona_andalucia,zona_aragon,zona_asturias,zona_baleares,zona_canarias,zona_cantabria,zona_castilla_la_mancha,zona_castilla_y_leon,zona_cataluña,zona_ceuta,zona_com_val,zona_extremadura,zona_galicia,zona_larioja,zona_madrid,zona_melilla,zona_murcia,zona_navarra,zona_pais_vasco,Prob_infection_initial,Prob_sintomatologia,user_id,Classification
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,63.4350,0.00,0,Sospechoso_sin_sintomas
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,71.5730,27.95,1,Sospechoso_sin_sintomas
2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,70.9975,21.53,2,Sospechoso_sin_sintomas
3,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,58.6430,49.48,3,Sospechoso_con_sintomas
4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,70.3750,12.11,4,Sospechoso_sin_sintomas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1572859,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,61.8315,87.89,1572859,Sospechoso_con_sintomas
1572860,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,37.6975,50.52,1572860,Sospechoso_con_sintomas
1572861,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,52.2460,78.47,1572861,Sospechoso_con_sintomas
1572862,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,48.9175,72.05,1572862,Sospechoso_con_sintomas


#Split in Train/Validación/Test



We will use the usually recommended ratios as an example:

Train: 70%.

Validation: 15%.

Test: 15%.



In [0]:
X=pd.DataFrame()

In [0]:
y = df['Classification']
X = df.loc[:, df.columns != 'Classification']
X = X.loc[:, X.columns != 'tos_seca']
X = X.loc[:, X.columns != 'user_id']
X = X.loc[:, X.columns != 'contacto_directo_no']
X = X.loc[:, X.columns != 'transporte_no']
X = X.loc[:, X.columns != 'proteccion_no']
X = X.loc[:, X.columns != 'Prob_sintomatologia']
X = X.loc[:, X.columns != 'Prob_infection_initial']

In [9]:
X = X.apply(pd.to_numeric) 
X

,fiebre,fatiga,flemas_esputos,dif_resp,dolor_garganta,dolor_cabeza,dolor_muscular,escalofrios,nausea_vomito,cong_nasal,diarrea,expect_sangre,conjuntivitis,dermatitis,falta_gusto_olfato,proteccion_si,transporte_si,contacto_directo_si,exposicion_alta,exposicion_baja,exposicion_media,zona_andalucia,zona_aragon,zona_asturias,zona_baleares,zona_canarias,zona_cantabria,zona_castilla_la_mancha,zona_castilla_y_leon,zona_cataluña,zona_ceuta,zona_com_val,zona_extremadura,zona_galicia,zona_larioja,zona_madrid,zona_melilla,zona_murcia,zona_navarra,zona_pais_vasco
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1572859,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1572860,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1572861,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1572862,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [0]:
perc_values = [0.7, 0.15, 0.15];

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train_rand, X_valtest_rand, y_train_rand, y_valtest_rand = train_test_split(X, y, test_size=perc_values[1] + perc_values[2], random_state=1);

X_val_rand, X_test_rand, y_val_rand, y_test_rand = train_test_split(X_valtest_rand, y_valtest_rand, test_size= perc_values[2] / (perc_values[1] + perc_values[2]), random_state=1)

In [13]:
print('Train data size = ' + str(X_train_rand.shape))
print('Train target size = ' + str(y_train_rand.shape))
print('Validation data size = ' + str(X_val_rand.shape))
print('Validation target size = ' + str(y_val_rand.shape))
print('Test data size = ' + str(X_test_rand.shape))
print('Test target size = ' + str(y_test_rand.shape))

Train data size = (1101004, 40)
Train target size = (1101004,)
Validation data size = (235930, 40)
Validation target size = (235930,)
Test data size = (235930, 40)
Test target size = (235930,)


# RANDOM FOREST

1) Import model.

In this case we have to go outside of scikit-learn to apply Random Forest Classifier model.

In [0]:
from sklearn.ensemble import RandomForestClassifier

2) Import metric

In [0]:
from sklearn.metrics import roc_auc_score as auc;
from sklearn.metrics import confusion_matrix as confusion_matrix;
from sklearn.metrics import accuracy_score as acc;

3) Define the method

In [16]:
model = RandomForestClassifier(n_estimators = 55,  random_state = 1)

model

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=55,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

4) Call the fit method to train the model

In [17]:
start = timeit.default_timer()

model.fit(X_train_rand, np.array(y_train_rand))

stop = timeit.default_timer()
print('Time: ', stop - start) 

Time:  88.25594434999994


5) Call the predict method to generate the predictions.

In [18]:
start = timeit.default_timer()

pred_train = model.predict(X_train_rand)
pred_val = model.predict(X_val_rand)
pred_test = model.predict(X_test_rand)

stop = timeit.default_timer()
print('Time: ', stop - start) 

Time:  29.362766947999717


In [0]:


# Label test

lb = preprocessing.LabelBinarizer()
lb.fit(y_test_rand)

y_test_lb = lb.transform(y_test_rand)
val_lb = lb.transform(pred_test)

#Label train

lb.fit(y_train_rand)

y_train_lb = lb.transform(y_train_rand)
val_train_lb = lb.transform(pred_train)

#Label validation

lb.fit(y_val_rand)

y_val_lb = lb.transform(y_val_rand)
val_val_lb = lb.transform(pred_val)


6) Calculate metrics using the predictions obtained in the previous step.

In [0]:
auc_train = auc(y_train_lb, val_train_lb, average='macro');
auc_val = auc(y_val_lb, val_val_lb, average='macro');
auc_test = auc(y_test_lb, val_lb, average='macro');
results = pd.DataFrame()
results = results.append(pd.DataFrame(data={'model':['Random Forest 1 Infection'],'auc_train':[auc_train],'auc_val':[auc_val],'auc_test':[auc_test]}, columns=['model',  'auc_train','auc_val', 'auc_test']), ignore_index=True)

In [21]:
results

,model,auc_train,auc_val,auc_test
0,Random Forest 1 Infection,0.991716,0.733353,0.732463


In [0]:
acc_train = acc(y_train_lb, val_train_lb);
acc_val = acc(y_val_lb, val_val_lb);
acc_test = acc(y_test_lb, val_lb);
results2 = pd.DataFrame()
results2 = results2.append(pd.DataFrame(data={'model':['Random Forest 1 Infection'],'acc_train':[acc_train],'acc_val':[acc_val],'acc_test':[acc_test]}, columns=['model',  'acc_train','acc_val', 'acc_test']), ignore_index=True)

In [23]:
results2

,model,acc_train,acc_val,acc_test
0,Random Forest 1 Infection,0.993532,0.800992,0.799886


## Importance of features

In [0]:
importances = list(model.feature_importances_)

In [0]:
# tuplas de importancia y variable
feature_importances = [(feature, round(importance, 3)) for feature, importance in zip(X, importances)]
# Ordenamos las variables por importancia
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)


In [26]:
[print(pair) for pair in feature_importances];

('fiebre', 0.226)
('contacto_directo_si', 0.15)
('fatiga', 0.047)
('flemas_esputos', 0.042)
('conjuntivitis', 0.033)
('dermatitis', 0.033)
('falta_gusto_olfato', 0.033)
('expect_sangre', 0.032)
('diarrea', 0.027)
('proteccion_si', 0.026)
('cong_nasal', 0.025)
('transporte_si', 0.025)
('nausea_vomito', 0.023)
('dif_resp', 0.021)
('dolor_cabeza', 0.02)
('dolor_muscular', 0.019)
('escalofrios', 0.019)
('dolor_garganta', 0.018)
('exposicion_alta', 0.01)
('exposicion_media', 0.01)
('exposicion_baja', 0.009)
('zona_andalucia', 0.008)
('zona_aragon', 0.008)
('zona_asturias', 0.008)
('zona_baleares', 0.008)
('zona_canarias', 0.008)
('zona_cantabria', 0.008)
('zona_castilla_la_mancha', 0.008)
('zona_castilla_y_leon', 0.008)
('zona_ceuta', 0.008)
('zona_com_val', 0.008)
('zona_extremadura', 0.008)
('zona_galicia', 0.008)
('zona_larioja', 0.008)
('zona_melilla', 0.008)
('zona_murcia', 0.008)
('zona_navarra', 0.008)
('zona_pais_vasco', 0.008)
('zona_cataluña', 0.007)
('zona_madrid', 0.007)


In [31]:
param_grid = [{'bootstrap': [True, False],
  'n_estimators': [10, 70, 100], 'max_features': [None, 0.01],'min_samples_leaf': [1, 2, 4],
  'max_depth': [10,  100, None]}
 ]

param_grid

[{'bootstrap': [True, False],
  'max_depth': [10, 100, None],
  'max_features': [None, 0.01],
  'min_samples_leaf': [1, 2, 4],
  'n_estimators': [10, 70, 100]}]

In [32]:
params_values = param_grid[0]
params_values

{'bootstrap': [True, False],
 'max_depth': [10, 100, None],
 'max_features': [None, 0.01],
 'min_samples_leaf': [1, 2, 4],
 'n_estimators': [10, 70, 100]}

In [33]:
num_iteraciones = len(params_values['bootstrap'])*len(params_values['n_estimators'])*len(params_values['max_features'])*len(params_values['min_samples_leaf'])*len(params_values['max_depth'])

print('El numero de iteraciones es', num_iteraciones)

El numero de iteraciones es 108


In [0]:
metric = acc

error_val= pd.DataFrame(columns=('val_error', 'bootstrap', 'n_estimators', 'max_features', 'min_samples_leaf', 'max_depth'))
num_iter = 0

In [37]:
start = timeit.default_timer()

for a in range(0,len(params_values['bootstrap'])):
    for b in range(0,len(params_values['n_estimators'])):
        for c in range(0,len(params_values['max_features'])):
            for d in range(0,len(params_values['min_samples_leaf'])):
                for e in range(0,len(params_values['max_depth'])):
                    # print control iteracion modelo
                    print('Numero de iteración = ' + str(num_iter) +
                          ', parametro boostrap = ' + str(params_values['bootstrap'][a]) + 
                          ', parametro number of trees = '  + str(params_values['n_estimators'][b]) +
                          ', parametro maximo de variables = '  + str(params_values['max_features'][c]) +
                          ', parametro minumun sample = '  + str(params_values['min_samples_leaf'][d]) +
                          ', parametro profundidad maxima = '  + str(params_values['max_depth'][e]))

                    # definicion del modelo con sus parametros
                    model = RandomForestClassifier(bootstrap = params_values['bootstrap'][a], 
                                                  n_estimators = params_values['n_estimators'][b],
                                                  max_features = params_values['max_features'][c],
                                                  min_samples_leaf = params_values['min_samples_leaf'][d],
                                                  max_depth = params_values['max_depth'][e], 
                                                  random_state = 1)

                    # entrenamiento del modelo
                    model.fit(X_train_rand, np.array(y_train_rand))

                    # prediccion del conjunto de validacion
                    pred_val = model.predict(X_val_rand)

                    # Calculo de la metrica de error
                    error_val_iter = metric(y_val_rand, pred_val)

                    # print error
                    print('Error validacion = ' + str(error_val_iter))

                    # guarda el error
                    error_val.loc[num_iter]=[error_val_iter,
                                            params_values['bootstrap'][a], 
                                            params_values['n_estimators'][b],
                                            params_values['max_features'][c],
                                            params_values['min_samples_leaf'][d],
                                            params_values['max_depth'][e]] 
                    num_iter += 1

stop = timeit.default_timer()
print('Time: ', stop - start) 

Numero de iteración = 0, parametro boostrap = True, parametro number of trees = 10, parametro maximo de variables = None, parametro minumun sample = 1, parametro profundidad maxima = 10
Error validacion = 0.8243165345653372
Numero de iteración = 1, parametro boostrap = True, parametro number of trees = 10, parametro maximo de variables = None, parametro minumun sample = 1, parametro profundidad maxima = 100
Error validacion = 0.8074428856016616
Numero de iteración = 2, parametro boostrap = True, parametro number of trees = 10, parametro maximo de variables = None, parametro minumun sample = 1, parametro profundidad maxima = None
Error validacion = 0.8074428856016616
Numero de iteración = 3, parametro boostrap = True, parametro number of trees = 10, parametro maximo de variables = None, parametro minumun sample = 2, parametro profundidad maxima = 10
Error validacion = 0.8243165345653372
Numero de iteración = 4, parametro boostrap = True, parametro number of trees = 10, parametro maximo 

In [38]:
error_val

,val_error,bootstrap,n_estimators,max_features,min_samples_leaf,max_depth
0,0.824317,True,10,NaN,1,10
1,0.807443,True,10,NaN,1,100
2,0.807443,True,10,NaN,1,None
3,0.824317,True,10,NaN,2,10
4,0.802903,True,10,NaN,2,100
...,...,...,...,...,...,...
103,0.813597,False,100,0.01,2,100
104,0.813597,False,100,0.01,2,None
105,0.754681,False,100,0.01,4,10
106,0.816636,False,100,0.01,4,100


In [45]:
ind_max = error_val['val_error'].idxmax()
best_parameters = error_val.iloc[ind_max]
params_values['max_depth']

[10, 100, None]

4) Call the fit method to train the model with best parameters

In [46]:
import math
if math.isnan(best_parameters['max_features']):
    best_parameters['max_features'] = None
model = RandomForestClassifier(bootstrap = best_parameters['bootstrap'], 
                                                  n_estimators = best_parameters['n_estimators'],
                                                  max_features = best_parameters['max_features'],
                                                  min_samples_leaf = best_parameters['min_samples_leaf'],
                                                  max_depth = best_parameters['max_depth'],
                                                  random_state = 1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [47]:
start = timeit.default_timer()

model.fit(X_train_rand, np.array(y_train_rand))

stop = timeit.default_timer()
print('Time: ', stop - start) 

Time:  56.76590360699993


In [51]:
start = timeit.default_timer()

pred_train = model.predict(X_train_rand)
pred_val = model.predict(X_val_rand)
pred_test = model.predict(X_test_rand)

stop = timeit.default_timer()
print('Time: ', stop - start) 

Time:  2.202215238001372


6) Calculate metrics using the predictions obtained in the previous step.

In [0]:
# Label test

lb = preprocessing.LabelBinarizer()
lb.fit(y_test_rand)

y_test_lb = lb.transform(y_test_rand)
val_lb = lb.transform(pred_test)

#Label train

lb.fit(y_train_rand)

y_train_lb = lb.transform(y_train_rand)
val_train_lb = lb.transform(pred_train)

#Label validation

lb.fit(y_val_rand)

y_val_lb = lb.transform(y_val_rand)
val_val_lb = lb.transform(pred_val)

In [0]:
auc_train = auc(y_train_lb, val_train_lb, average='macro');
auc_val = auc(y_val_lb, val_val_lb, average='macro');
auc_test = auc(y_test_lb, val_lb, average='macro');
results = pd.DataFrame()
results = results.append(pd.DataFrame(data={'model':['Random Forest 1 Infection (Optimized)'],'auc_train':[auc_train],'auc_val':[auc_val],'auc_test':[auc_test]}, columns=['model',  'auc_train','auc_val', 'auc_test']), ignore_index=True)

In [54]:
results

,model,auc_train,auc_val,auc_test
0,Random Forest 1 Infection (Optimized),0.757737,0.751818,0.750515


In [0]:
acc_train = acc(y_train_lb, val_train_lb);
acc_val = acc(y_val_lb, val_val_lb);
acc_test = acc(y_test_lb, val_lb);
results2 = pd.DataFrame()
results2 = results2.append(pd.DataFrame(data={'model':['Random Forest 1 Infection (Optimized)'],'acc_train':[acc_train],'acc_val':[acc_val],'acc_test':[acc_test]}, columns=['model',  'acc_train','acc_val', 'acc_test']), ignore_index=True)

In [56]:
results2

,model,acc_train,acc_val,acc_test
0,Random Forest 1 Infection (Optimized),0.828801,0.824567,0.822952
